In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import matplotlib
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt 
import numpy as np
import seaborn as sn
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets, metrics, model_selection, svm
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

Upload Bob and Mel individual data


In [3]:
bob = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Bob.csv')
mel = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Mel.csv')
#elab = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/BobandMellCDM.csv')




Extract only 'gaze' data from datasets

In [4]:
bob = bob.loc[:,bob.columns.str.contains('gaze')]
mel = mel.loc[:,mel.columns.str.contains('gaze')]

Create an 'Elaborated' column for both bob and mel

In [5]:
elabo = []
for i in range(97056):
  elabo.append(0)
bob['Elaborated'] = elabo
mel['Elaborated'] = elabo

I manually filled in the 'Elaborated' values for the first 15 minutes. The 'Elaborated' values in the CDM.csv file were on a completely different timestamp interval than the Bob and Mel datasets, so it would have been extremely difficult to write an algorithm to link them. Most of the 'Elaborated' data was in the first 15 minutes so I figured that was a good sample size to look at. 

In [6]:
mel= mel.to_numpy()
bob = bob.to_numpy()

In [7]:
for i in range(1050,1199):
  mel[i][8] = 1
for i in range(1260,1319):
  mel[i][8] = 1
for i in range(1230,1259):
  bob[i][8] = 1
for i in range(1440,1799):
  bob[i][8] = 1
for i in range(1980,2339):
  mel[i][8] = 1
for i in range(2370,2459):
  bob[i][8] = 1
for i in range(2520,2729):
  bob[i][8] = 1
for i in range(2970,3420):
  mel[i][8] = 1
for i in range(3421,3449):
  bob[i][8] = 1
for i in range(3510,3569):
  mel[i][8] = 1
for i in range(3630,3869):
  mel[i][8] = 1
for i in range(3869,3889):
  bob[i][8] = 1
for i in range(3990,4169):
  mel[i][8] = 1
for i in range(4320,4350):
  bob[i][8] = 1
for i in range(4350,4380):
  mel[i][8] = 1
for i in range(5010,5040):
  mel[i][8] = 1
for i in range(5310,5370):
  bob[i][8] = 1
for i in range(5310,5490):
  mel[i][8] = 1
for i in range(5580,5700):
  bob[i][8] = 1
for i in range(5700,5730):
  mel[i][8] = 1
for i in range(6060,6540):
  mel[i][8] = 1
for i in range(6540,7140):
  bob[i][8] = 1
for i in range(7140,7170):
  mel[i][8] = 1
for i in range(7350,7410):
  bob[i][8] = 1
for i in range(7410,7500):
  mel[i][8] = 1
for i in range(7500,7740):
  bob[i][8] = 1
for i in range(7740,7860):
  mel[i][8] = 1
for i in range(8010,8070):
  bob[i][8] = 1
for i in range(8070,8190):
  mel[i][8] = 1
for i in range(8190,8250):
  bob[i][8] = 1
for i in range(8250,8370):
  mel[i][8] = 1
for i in range(8370,8460):
  mel[i][8] = 1
for i in range(8460,8520):
  bob[i][8] = 1
for i in range(8520,8580):
  mel[i][8] = 1
for i in range(8850,8940):
  mel[i][8] = 1
for i in range(8940,9000):
  bob[i][8] = 1
for i in range(9000,9060):
  mel[i][8] = 1
for i in range(9060,9180):
  bob[i][8] = 1
for i in range(9180,9240):
  mel[i][8] = 1
for i in range(9240,9300):
  bob[i][8] = 1
for i in range(9300,9600):
  mel[i][8] = 1
for i in range(10130,10340):
  mel[i][8] = 1
for i in range(10340,10560):
  bob[i][8] = 1
for i in range(16290,16620):
  bob[i][8] = 1
for i in range(16620,16800):
  mel[i][8] = 1
for i in range(16920,17070):
  bob[i][8] = 1
for i in range(17070,17100):
  mel[i][8] = 1
for i in range(17400,17490):
  bob[i][8] = 1
for i in range(17670,17790):
  bob[i][8] = 1
for i in range(18150,18330):
  mel[i][8] = 1
for i in range(19230,19590):
  mel[i][8] = 1
for i in range(19770,19920):
  bob[i][8] = 1
for i in range(19920,20100):
  mel[i][8] = 1
for i in range(20370,21000):
  bob[i][8] = 1

In [8]:
mel2 = []
bob2 = []
mel2[:21000] = mel[:21000]
bob2[:21000] = bob[:21000]

Total number of elaborations

In [9]:
elabs = 0
for i in range(len(mel2)):
  if mel2[i][8] == 1:
    elabs += 1
  if bob2[i][8] == 1:
    elabs += 1
elabs

8478

lookElabs = number of elaborations during joint attention
look = number of time intervals with joint attention

In [10]:
look = 0
lookElab = 0
for i in range(len(mel2)):
  if (mel2[i][0] + bob2[i][0] >= -.2) and (mel2[i][0] + bob2[i][0] <= .2) and (mel2[i][1] + bob2[i][1] >= -.2) and (mel2[i][1] + bob2[i][1] <= .2):
    look += 1
    if mel2[i][8] == 1:
      lookElab += 1
    if bob2[i][8] == 1:
      lookElab += 1
print(look, lookElab, elabs)

1042 596 8478


57.2% of the time Bob and Mel likely have joint attention in the first 15 minutes of interaction, there is an Elaboration. 

In [11]:
nolook = 0
nolookElab = 0
for i in range(len(mel2)):
  if (mel2[i][0] + bob2[i][0] < -.2) or (mel2[i][0] + bob2[i][0] > .2) or (mel2[i][1] + bob2[i][1] < -.2) or (mel2[i][1] + bob2[i][1] > .2):
    nolook += 1
    if mel2[i][8] == 1:
      nolookElab += 1
    if bob2[i][8] == 1:
      nolookElab += 1
print(nolook, nolookElab, elabs)

19958 7882 8478


39.6% of the time Bob and Mel are looking away from each other, there is an Elaboration. Considering how this is nearly 20% than with joint attention, there is likely some correlation between gaze_x, gaze_y and 'Elaborated' values. 